# Création des resources RDF des mentions de parcelles

In [98]:
import json 
import pandas as pd
import numpy as np
import uuid
import re
from datetime import datetime
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS
from functions import *

## 1. Lecture des données

### Articles de classement

In [99]:
COMMUNE = 'Gentilly'
matrices_metada = {
    "MAT_1813": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_B_NB_1813",
        "MATRICE_START": "1813",
        "MATRICE_END": "1835"
    },
    "MAT_1836": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_NB_1836",
        "MATRICE_START": "1836",
        "MATRICE_END": "1847"
    },
    "MAT_1848": {
        "PLAN": "1845",
        "MATRICE_ID": "MAT_NB_1848",
        "MATRICE_START": "1848",
        "MATRICE_END": "1859"
    }
}


In [100]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/

PATH = ROOT + "data/gentilly/MAT_1813.csv"
mat1813 = pd.read_csv(PATH,header=0)
PATH = ROOT + "data/gentilly/MAT_1836.csv"
mat1836 = pd.read_csv(PATH,header=0)
PATH = ROOT + "data/gentilly/MAT_1848.csv"
mat1848 = pd.read_csv(PATH,header=0)

OUTPUT_FOLDER_PATH = ROOT + "data/rdf"

In [101]:
mat1813['registre'] = 'MAT_1813'
mat1836['registre'] = 'MAT_1836'
mat1848['registre'] = 'MAT_1848'

matrices = pd.concat([mat1813, mat1836, mat1848])
matrices = matrices.reset_index(drop=True)
print(matrices.columns)

Index(['ID', 'UUID', 'Type_CF', 'Num_Folio', 'Alt_Num_CF', 'Groupe_CF',
       'Ordre_de_lecture', 'Voie', 'Num_Voie', 'Image', 'Section_clean',
       'Parcelle_clean', 'Parcelle_treated', 'Lieu-dit_transcript',
       'Lieu-dit_clean', 'Lieu-dit_treated', 'Lieu-dit_type',
       'Propriétaires_transcript', 'Nature_transcript', 'Nature_clean',
       'Nature_treated', 'Date entrée', 'Date entrée_treated', 'Date sortie',
       'Date sortie_treated', 'Tiré de', 'Tiré de_treated', 'Porté à',
       'Porté à_treated', 'Ligne barrée ?', 'CF rayé ?', 'Spécification',
       'Commentaire', 'Cote liée', 'registre'],
      dtype='object')


In [102]:
display(matrices)

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe_CF,Ordre_de_lecture,Voie,Num_Voie,Image,...,Tiré de,Tiré de_treated,Porté à,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre
0,1,7ab9a942-8c75-45f2-a04f-7466a762ddbb,Bâti,108,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0586,...,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813
1,2,afdad904-966f-4ecf-8e09-cdae95dddc28,Bâti,114,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0588,...,NaN,NaN,82,82,Oui,Oui,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813
2,3,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,Bâti,82,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0579,...,114,114,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813
3,4,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,Bâti,82,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813
4,5,5f26bd83-015b-416c-a62e-4e33cb016560,Bâti,64,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0571,...,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,219,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,Non Bâti,1221,NaN,1,6,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,NaN,NaN,NaN,Non,Non,NaN,NaN,NaN,MAT_1848
438,220,0757ae64-497c-40b7-a1db-3d4730e71368,Non Bâti,1221,NaN,1,7,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,NaN,NaN,NaN,Non,Non,NaN,NaN,NaN,MAT_1848
439,221,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,Non Bâti,1222,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,NaN,NaN,NaN,Non,Non,NaN,NaN,NaN,MAT_1848
440,222,e7b76451-15e1-4c7e-b84e-a4336fb83654,Non Bâti,1222,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,NaN,NaN,NaN,Non,Non,NaN,NaN,NaN,MAT_1848


### Propriétaires

In [103]:
#open three json files
with open(ROOT + "data/gentilly/output_structured_owners.json") as f:
    data_owners = json.load(f)

#read as df
owners_df = pd.DataFrame(data_owners)

In [104]:
oregistre = []
otype_folio = []
ofolio = []
o_groupe_cf = []
o_transcription = []

for row in owners_df.iterrows():
    cell_info = row[1]['cell']
    oregistre.append(cell_info['registre'])
    otype_folio.append(cell_info['type_folio'])
    ofolio.append(cell_info['folio'])
    o_groupe_cf.append(cell_info['groupe_cf'])
    o_transcription.append(cell_info['transcription'])

owners_df['registre'] = oregistre
owners_df['type_folio'] = otype_folio
owners_df['folio'] = ofolio
owners_df['groupe_cf'] = o_groupe_cf
owners_df['transcription'] = o_transcription


In [105]:
owners_df

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris"
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...
4,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Bacot', 'o...",[],MAT_1813,Non Bâti,11,1,Bacot→(david) couverturier→à Paris
...,...,...,...,...,...,...,...,...
185,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...","[{'change-order': 1, 'owner-before': 1, 'owner...",MAT_1848,Non Bâti,1218,1,~~Delon~~ 1858 Sevestre Clément boulanger Bar↑...
186,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...",[],MAT_1848,Non Bâti,1219,1,Delon
187,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Dufresne',...",[],MAT_1848,Non Bâti,1220,1,Dufresne
188,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul


### 1.1. Création des pages
- rdf:type rico:Instanciation : instance numérisée d'une page de registre
- rdf:type rico:Record => concept de la page de registre, fait le lien avec le registre (concept, RecordSet)

In [106]:
#select distinct values in th colum Image
images = matrices[['registre','Image']].drop_duplicates().reset_index(drop=True)
images

,registre,Image
0,MAT_1813,FRAD094_3P_000255_01_0586
1,MAT_1813,FRAD094_3P_000255_01_0588
2,MAT_1813,FRAD094_3P_000255_01_0579
3,MAT_1813,FRAD094_3P_000255_01_0571
4,MAT_1813,FRAD094_3P_000255_01_0015
...,...,...
150,MAT_1848,FRAD094_3P_000264_01_0218
151,MAT_1848,FRAD094_3P_000264_01_0219
152,MAT_1848,FRAD094_3P_000264_01_0220
153,MAT_1848,FRAD094_3P_000264_01_0221


In [107]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('cad', cad)
g.bind('add', add)
g.bind('source', srcuri)
g.bind('mlclasse', mlclasse)
g.bind('activity', activity)

g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

for index, row in images.iterrows():
    img = row['Image']
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
    json = parse_record_id(img)
    subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{img}")
    g.add((subject_uri, RDF.type, rico.Instanciation))
    g.add((subject_uri, rico.identifier, Literal(img)))
    folder_end = img.rfind('_')
    #Record (Page)
    subject_uri_record = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{img}_page")
    g.add((subject_uri_record, RDF.type, rico.Record))
    g.add((subject_uri,rico.isOrWasDigitalInstanciationOf,subject_uri_record))
    #Record (Page) se trouve dans une matrice donnée (record)
    g.add((subject_uri_record, rico.isOrWasIncludedIn, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))

    # Activité 0001 : Classification des pages
    mlClasseNode = BNode()
    g.add((subject_uri, cad.hasClasse, mlClasseNode))
    g.add((mlClasseNode, cad.hasClasseValue, URIRef(mlclasse + f"MATMainTable")))
    g.add((mlClasseNode, PROV.wasGeneratedBy, URIRef(activity + f"0001")))
    g.add((URIRef(activity + "0001"), PROV.used, URIRef(subject_uri)))

    #Activité 0002 : Extraction des informations des pages
    g.add((URIRef(activity + "0002"), PROV.used, URIRef(subject_uri)))

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_pages.ttl", format='turtle')

@prefix activity: <http://data.ign.fr/id/codes/cadastre/activity/> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .

activity:0002 prov:used source:94_Gentilly_FRAD094_3P_000255_01_0015,
        source:94_Gentilly_FRAD094_3P_000255_01_0032,
        source:94_Gentilly_FRAD094_3P_000255_01_0044,
        source:94_Gentilly_FRAD094_3P_000255_01_0057,
        source:94_Gentilly_FRAD094_3P_000255_01_0125,
        source:94_Gentilly_FRAD094_3P_000255_01_0146,
        source:94_Gentilly_FRAD094_3P_000255_01_0148,
        source:94_Gentilly_FRAD094_3P_000255_01_0165,
        source:94_Gentilly_FRAD094_3P_000255_01_0186,
        source:94_Gentilly_FRAD094_3P_000255_01_0196,
        source:94_Gentilly_FRAD094_3P_000255_01_0199,
        source:94_Gentilly_FRAD094_3P_000255

<Graph identifier=Nc8638afc4a7e4b97b24bb1ecaf4b80ba (<class 'rdflib.graph.Graph'>)>

### 1.2 Folios
- Pré-traitement des colonnes *Num_Folio*, *Tiré de* et *Porté à*
- Création des objets "Folios" à partir de la colonne *Num_Folio* et des colonnes *Tiré de* et *Porté à* (manquants)
- Création des objets spéciaux mentionnés dans les colonnes destinées aux folios (reste, construction nouvelle, ruine etc)

#### Pré-traitement

In [108]:
from functions import parse_record_id, cleanNumFolio

#Clean columns Num_Folio, Tire_de, Porte_a
clean_folio, clean_tire_de, clean_porte_a = [], [], []
symbols = [",", "→", "."," ",";","&"]

for index, row in matrices.iterrows():
    clean_folio.append(cleanNumFolio(row["Num_Folio"],symbols))
    clean_tire_de.append(cleanNumFolio(row["Tiré de_treated"],symbols))
    clean_porte_a.append(cleanNumFolio(row["Porté à_treated"],symbols))

# Create new columns containing the cleaned values
matrices['Num_Folio_clean'] = clean_folio
matrices['Tire_de_clean'] = clean_tire_de
matrices['Porte_a_clean'] = clean_porte_a

matrices['Num_Folio_clean'] = matrices['Num_Folio_clean'].astype(str)

['236↑4↓', '361', '258', '166', '235↑2↓', '138', '357', '440']
['248', 'additionconstructionsv']
['249', '249']
['288', '433']
['443', '443']
['450', '443', '453', '968']
['449', '968']
['968', '442']
['442', '449', '439', '450']
['836', '403']
['837', '403']
['403', '513↑25↓']
['837', 'additionconstructionsv']
['844', 'additionconstructionsv']
['844', 'additionconstructionsv']
['846↑2↓', 'additionconstructionsv']
['847↑2↓', 'additionconstructionsv']
['847↑2↓', 'additionconstructionsv']


In [109]:
print(clean_porte_a)

['EMPTY', '82', 'EMPTY', 'EMPTY', 'EMPTY', '156', '156', '192bis', '280bis', 'EMPTY', 'EMPTY', '34ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '124bis', 'EMPTY', 'EMPTY', '192bis', '247ter', 'voiepubliquesv', 'EMPTY', 'EMPTY', '46bis', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '211bis', '107↑2↓', 'EMPTY', 'doubleemploisv', 'EMPTY', 'EMPTY', '269↑2↓', '138', '236ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '361', 'EMPTY', 'EMPTY', 'EMPTY', '236↑4↓;361;258;166;235↑2↓;138;357;440', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '249', 'EMPTY', 'EMPTY', '248', 'EMPTY', '249;249', 'EMPTY', 'EMPTY', '249↑16↓', '249↑16↓', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'demolitionsv', 'EMPTY', 'EMPTY', 'demolitionsv', '288;433', 'EMPTY', 'EMPTY', 'EMPTY', 'augmentationsv', 'EMPTY', 'EMPTY', 'EMPTY

In [110]:
#Using matrices, create new df named folios containing all lines of matrices where register = MAT_1836 and MAT_1848. For register=MAT_1813, remove the lines where type_CF = "Bâti"
folios = matrices.copy()
folios.reset_index(drop=True)

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe_CF,Ordre_de_lecture,Voie,Num_Voie,Image,...,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean
0,1,7ab9a942-8c75-45f2-a04f-7466a762ddbb,Bâti,108,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0586,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,108,EMPTY,EMPTY
1,2,afdad904-966f-4ecf-8e09-cdae95dddc28,Bâti,114,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0588,...,82,Oui,Oui,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,114,EMPTY,82
2,3,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,Bâti,82,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,114,EMPTY
3,4,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,Bâti,82,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,EMPTY,EMPTY
4,5,5f26bd83-015b-416c-a62e-4e33cb016560,Bâti,64,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0571,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,64,EMPTY,EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,219,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,Non Bâti,1221,NaN,1,6,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
438,220,0757ae64-497c-40b7-a1db-3d4730e71368,Non Bâti,1221,NaN,1,7,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
439,221,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,Non Bâti,1222,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY
440,222,e7b76451-15e1-4c7e-b84e-a4336fb83654,Non Bâti,1222,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY


Il doit rester 437 lignes sur 442.

In [111]:
#Create a new dataframe from sources with columns "Num_Folio" and "Image" containing only distinct rows
folios_pages = folios[["Num_Folio","Num_Folio_clean","Alt_Num_CF","Image","registre","Type_CF"]].drop_duplicates(subset=["Num_Folio","Num_Folio_clean","Alt_Num_CF","Image","registre","Type_CF"]).reset_index(drop=True)
display(folios_pages)

,Num_Folio,Num_Folio_clean,Alt_Num_CF,Image,registre,Type_CF
0,108,108,NaN,FRAD094_3P_000255_01_0586,MAT_1813,Bâti
1,114,114,NaN,FRAD094_3P_000255_01_0588,MAT_1813,Bâti
2,82,82,NaN,FRAD094_3P_000255_01_0579,MAT_1813,Bâti
3,64,64,NaN,FRAD094_3P_000255_01_0571,MAT_1813,Bâti
4,11,11,NaN,FRAD094_3P_000255_01_0015,MAT_1813,Non Bâti
...,...,...,...,...,...,...
152,1218,1218,NaN,FRAD094_3P_000264_01_0218,MAT_1848,Non Bâti
153,1219,1219,NaN,FRAD094_3P_000264_01_0219,MAT_1848,Non Bâti
154,1220,1220,NaN,FRAD094_3P_000264_01_0220,MAT_1848,Non Bâti
155,1221,1221,NaN,FRAD094_3P_000264_01_0221,MAT_1848,Non Bâti


In [112]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('source', srcuri)
g.bind('srctype', srctype)
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in folios_pages.iterrows():
    json = parse_record_id(row['Image'])
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

    subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio'])}")
    g.add((subject_uri, RDF.type, rico.RecordPart))
    if row["registre"] == "MAT_1813" and row["Type_CF"] == "Bâti":
        g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioBati")))
    else:
        g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioNonBati")))
    #g.add((subject_uri, cad.isSourceType, URIRef(srctype.Folio)))
    g.add((subject_uri, cad.hasNumFolio, Literal(row["Num_Folio_clean"],datatype=XSD.string)))
    g.add((subject_uri, rico.isOrWasConstituentOf,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))

    if not pd.isna(row['Alt_Num_CF']):
        g.add((subject_uri, cad.hasAlternativeNumFolio, Literal(int(row["Alt_Num_CF"]),datatype=XSD.string)))

print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_MAT_B_NB_1813_107bis a rico:RecordPart ;
    cad:hasNumFolio "107bis"^^xsd:string ;
    cad:isSourceType srctype:FolioNonBati ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

source:94_Gentilly_MAT_B_NB_1813_108 a rico:RecordPart ;
    cad:hasNumFolio "108"^^xsd:string ;
    cad:isSourceType srctype:FolioBati ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0586_page .

source:94_Gentilly_MAT_B_NB_1813_11 a rico:RecordPart ;
    cad:hasNumFolio "11"^^xsd:string ;
    cad:isSourceType srctype:FolioNonBati ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:94_Gentilly_MAT_B_NB_1813_114 a rico

#### Création des folios issus de "Tiré de" et "Porté à" qui ne sont pas dans la colonne 'Num_Folios'

In [113]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Tire_de_clean'] != 'EMPTY':
        ls = row['Tire_de_clean'].split(";")
        for l in ls:
            if any(num.isdigit() for num in l) and 'mission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                if row["registre"] == "MAT_1813" and row["Type_CF"] == "Bâti":
                    g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioBati")))
                else:
                    g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioNonBati")))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [114]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Porte_a_clean'] != 'EMPTY':
        ls = row['Porte_a_clean'].split(";")
        for l in ls:
            #test if str has digit
            if any(num.isdigit() for num in l) and 'mission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                if row["registre"] == "MAT_1813" and row["Type_CF"] == "Bâti":
                    g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioBati")))
                else:
                    g.add((subject_uri, cad.isSourceType, URIRef(srctype + "FolioNonBati")))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [115]:
print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_folios.ttl", format='turtle')

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_MAT_B_NB_1813_107bis a rico:RecordPart ;
    cad:hasNumFolio "107bis"^^xsd:string ;
    cad:isSourceType srctype:FolioNonBati ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

<http://data.ign.fr/id/source/94_Gentilly_MAT_B_NB_1813_107↑2↓> a rico:RecordPart ;
    cad:hasNumFolio "107↑2↓"^^xsd:string ;
    cad:isSourceType srctype:FolioNonBati .

source:94_Gentilly_MAT_B_NB_1813_108 a rico:RecordPart ;
    cad:hasNumFolio "108"^^xsd:string ;
    cad:isSourceType srctype:FolioBati ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0586_page .

source:94_Gentilly_MAT_B_NB_1813_11 a rico:RecordPart ;
    cad:hasNumFolio "11"^^xsd:string 

<Graph identifier=Nb7e9662938e1490396536a2b61f79cda (<class 'rdflib.graph.Graph'>)>

### 1.3 Adresses

In [116]:
# Select the column as a new DataFrame
addresses = matrices[['registre','Lieu-dit_treated','Lieu-dit_type']].copy().drop_duplicates().reset_index(drop=True)
addresses

,registre,Lieu-dit_treated,Lieu-dit_type
0,MAT_1813,NaN,NaN
1,MAT_1813,Les Girantiers,District
2,MAT_1813,Rue Thiers,Thoroughfare
3,MAT_1813,Les Girantins,District
4,MAT_1813,Rue Thiers;10,Address
...,...,...,...
130,MAT_1848,Barrière d'Italie;8↑B↓,Address
131,MAT_1848,Barrière d'Italie;8↑A↓,Address
132,MAT_1848,Barrière d'Italie;6,Address
133,MAT_1848,Barrière d'Italie;4,Address


In [117]:
multipart_addresses_street = []
multipart_addresses_street_number = []
multipart_addresses_street_type = []
multipart_addresses_street_number_type = []


for index, row in addresses.iterrows():
    tag = str(row["Lieu-dit_treated"])
    if ';' in tag:
        add = tag.split(";")
        add_street_or_district = add[0]
        add_num_or_part = add[1]

        multipart_addresses_street.append(add_street_or_district)
        multipart_addresses_street_number.append(add_num_or_part)

        #test if digit
        if any(num.isdigit() for num in add_num_or_part):
            multipart_addresses_street_type.append('Thoroughfare')
            multipart_addresses_street_number_type.append('StreetNumber')
        else:
            multipart_addresses_street_type.append('District')
            multipart_addresses_street_number_type.append('Undefined')

    else:
        multipart_addresses_street.append('')
        multipart_addresses_street_number.append('')
        multipart_addresses_street_type.append('')
        multipart_addresses_street_number_type.append('')

addresses['part_street_district'] = multipart_addresses_street
addresses['part_street_number'] = multipart_addresses_street_number
addresses['part_street_district_type'] = multipart_addresses_street_type
addresses['part_street_number_type'] = multipart_addresses_street_number_type

#assign a distinct uuid for each group of rows with same values in Lieu-dit_treated and registre
addresses['address_uuid'] = [uuid.uuid4() for _ in range(len(addresses))]

addresses

,registre,Lieu-dit_treated,Lieu-dit_type,part_street_district,part_street_number,part_street_district_type,part_street_number_type,address_uuid
0,MAT_1813,NaN,NaN,,,,,8e972aea-7b15-41d5-bd9b-7050d885c1cd
1,MAT_1813,Les Girantiers,District,,,,,c67bb76c-5ccf-48a0-93a5-54e6d6b398f9
2,MAT_1813,Rue Thiers,Thoroughfare,,,,,eb6241fa-8074-4313-985e-b932312d630a
3,MAT_1813,Les Girantins,District,,,,,bd3c3b01-bc85-4196-a661-6ec3e8ce59df
4,MAT_1813,Rue Thiers;10,Address,Rue Thiers,10,Thoroughfare,StreetNumber,f1815deb-fa7f-4013-8a94-e7650442518e
...,...,...,...,...,...,...,...,...
130,MAT_1848,Barrière d'Italie;8↑B↓,Address,Barrière d'Italie,8↑B↓,Thoroughfare,StreetNumber,ff91c792-397d-4931-9859-cf9b74d4450d
131,MAT_1848,Barrière d'Italie;8↑A↓,Address,Barrière d'Italie,8↑A↓,Thoroughfare,StreetNumber,bbefdc00-b53d-4f85-81f9-8344e51d2baf
132,MAT_1848,Barrière d'Italie;6,Address,Barrière d'Italie,6,Thoroughfare,StreetNumber,dd8c28e8-1c8a-4613-929b-fa2d64656618
133,MAT_1848,Barrière d'Italie;4,Address,Barrière d'Italie,4,Thoroughfare,StreetNumber,51a78e91-1ba4-4236-9f39-ed9b796f677e


In [118]:
import pandas as pd
import uuid

# Create a new column 'part_street_district_uuid'
addresses['part_street_district_uuid'] = None

# Create a dictionary to store the uuid for each unique part_street_district
uuid_dict = {}

# Iterate over the DataFrame
for i, row in addresses.iterrows():
    if row['part_street_district'] != '':
        # Check if the part_street_district value is equal to one of the Lieu-dit_treated values
        if row['part_street_district'] in addresses['Lieu-dit_treated'].values:
            # If yes, set the part_street_district_uuid to the uuid of the retrieved Lieu-dit_treated
            addresses.loc[i, 'part_street_district_uuid'] = addresses.loc[addresses['Lieu-dit_treated'] == row['part_street_district'], 'address_uuid'].values[0]
        else:
            # If no, check if the part_street_district value has other occurrences in the part_street_district column
            if row['part_street_district'] in addresses['part_street_district'].values:
                # If yes, check if the part_street_district value is already in the uuid_dict
                if row['part_street_district'] in uuid_dict:
                    # If yes, assign the same uuid
                    addresses.loc[i, 'part_street_district_uuid'] = uuid_dict[row['part_street_district']]
                else:
                    # If no, create a new uuid and add it to the uuid_dict
                    new_uuid = uuid.uuid4()
                    uuid_dict[row['part_street_district']] = new_uuid
                    addresses.loc[i, 'part_street_district_uuid'] = new_uuid
            else:
                # If no, create a new uuid
                addresses.loc[i, 'part_street_district_uuid'] = uuid.uuid4()

In [119]:
uuid_dict

{'Rue Mazagran': UUID('2ff7909f-3546-4731-ad19-4c1753300e18')}

In [120]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('source', srcuri)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

g.bind("ltype", ltype)
g.bind("lrtype", lrtype)
g.bind("atype", atype)
g.bind('cad_ltype', cad_ltype)

for index, row in addresses.iterrows():
    add_uri = URIRef(landmarkuri + f"{row['address_uuid']}")
    g.add((add_uri, RDF.type, add.Landmark))
    if pd.notnull(addresses.loc[index, 'Lieu-dit_type']):
        if ';' in row['Lieu-dit_treated']:
            name = row['Lieu-dit_treated'].split(";")

            g.add((add_uri, add.isLandmarkType, URIRef(ltype + row['part_street_number_type'])))

            if any(num.isdigit() for num in name[1]):
                g.add((add_uri, SKOS.prefLabel, Literal(name[1] + ' (' + name[0] + ', ' + COMMUNE + ')', datatype=XSD.string)))
                g.add((add_uri, SKOS.altLabel, Literal(name[1] + ' ' + name[0], datatype=XSD.string)))
                g.add((add_uri,SKOS.hiddenLabel,Literal(name[1],datatype=XSD.string))) #Street number from transcription
                relationode = BNode()
                g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Along))
            else:
                g.add((add_uri, SKOS.prefLabel, Literal(name[0] + ' (' + name[1] + ', ' + COMMUNE + ')', datatype=XSD.string)))
                g.add((add_uri, SKOS.altLabel, Literal(name[0], datatype=XSD.string)))
                relationode = BNode()
                g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Undefined))

            g.add((URIRef(relationode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(relationode.n3()), add.locatum, add_uri))
            g.add((URIRef(relationode.n3()), add.relatum, URIRef(landmarkuri + str(row['part_street_district_uuid']))))

            #Street or district relation with section
            sectionNode = BNode()
            g.add((URIRef(sectionNode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(sectionNode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(sectionNode.n3()), add.locatum, URIRef(landmarkuri + str(row['part_street_district_uuid']))))
            if row['registre'] != 'MAT_1848':
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + 'da6a5c2c-e86d-43bb-8950-7169bd0df60a'))) #Section D Cadastre 1848
            else:
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + '87d7c2f6-306b-45a1-a833-5e17821c3102'))) #Section B Cadastre 1811

        else:
            g.add((add_uri, add.isLandmarkType, URIRef(ltype + f"{row['Lieu-dit_type']}")))
            g.add((add_uri, SKOS.prefLabel, Literal(row['Lieu-dit_treated'] + ', ' + COMMUNE, datatype=XSD.string)))
            g.add((add_uri, SKOS.altLabel, Literal(row['Lieu-dit_treated'], datatype=XSD.string)))

            relationode = BNode()
            g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(relationode.n3()), add.locatum, add_uri))

            sectionNode = BNode()
            g.add((URIRef(sectionNode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(sectionNode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(sectionNode.n3()), add.locatum, add_uri))
            if row['registre'] != 'MAT_1848':
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + 'da6a5c2c-e86d-43bb-8950-7169bd0df60a'))) #Section D Cadastre 1848
            else:
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + '87d7c2f6-306b-45a1-a833-5e17821c3102'))) #Section B Cadastre 1811

        g.add((add_uri, cad.sourcedFrom, URIRef(srcuri + f'94_{COMMUNE}_{row["registre"]}')))

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_landmarks_lieu_dit.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/> .
@prefix ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N0007df1894854bc4875e43370ca9a161> a add:LandmarkRelation ;
    add:isLandmarkRelationType lrtype:Within ;
    add:locatum landmark:eb6241fa-8074-4313-985e-b932312d630a ;
    add:relatum landmark:87d7c2f6-306b-45a1-a833-5e17821c3102 .

<_:N000dadaa8f9b482fbac94cf74786d52f> a add:LandmarkRelation ;
    add:isLandmarkRelationType lrtype:Within ;
    add:locatum landmark:2ff7909f-3546-4731-ad19-4c1753300e18 ;
    add:relatum landmark:da6a5c2c-e86d-43bb-8950-7169bd0df60a .

<_:N011cb71161954dd38ea

<Graph identifier=N6cb369df748842f983286e53ccf79a39 (<class 'rdflib.graph.Graph'>)>

### 1.5 Propriétaires

In [121]:
from pandasql import sqldf
import pandas as pd

In [122]:
matrices_ = matrices[['registre','Num_Folio','Type_CF','Groupe_CF','Image','UUID','Ordre_de_lecture']].copy()

In [123]:
#add column with rows index
owners_df['row_index'] = owners_df.index
owners_df2 = owners_df[['registre','type_folio','folio','groupe_cf','row_index']]
owners_df2.dtypes

registre      object
type_folio    object
folio         object
groupe_cf     object
row_index      int64
dtype: object

In [124]:
query = '''SELECT M.registre AS m_registre, M.Type_CF AS m_type_folio, M.Num_Folio AS m_num_folio, M.Groupe_CF AS m_groupe_cf, M.Image AS m_image, M.UUID AS m_uuid, m.Ordre_de_lecture AS m_row_num_in_cf, O.row_index AS o_row_index
        FROM matrices_ AS M
        LEFT JOIN owners_df2 AS O
        ON (M.Num_Folio = O.folio AND M.registre = O.registre AND M.Groupe_CF = O.groupe_cf AND M.Type_CF = O.type_folio)
        '''

testDF = sqldf(query)
#testDF.to_csv(ROOT + 'test.csv',index=False)
#pd.set_option("display.max_rows", None)
display(sqldf(query))

,m_registre,m_type_folio,m_num_folio,m_groupe_cf,m_image,m_uuid,m_row_num_in_cf,o_row_index
0,MAT_1813,Bâti,108,1,FRAD094_3P_000255_01_0586,7ab9a942-8c75-45f2-a04f-7466a762ddbb,1,0
1,MAT_1813,Bâti,114,1,FRAD094_3P_000255_01_0588,afdad904-966f-4ecf-8e09-cdae95dddc28,1,1
2,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,1,2
3,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,2,2
4,MAT_1813,Bâti,64,1,FRAD094_3P_000255_01_0571,5f26bd83-015b-416c-a62e-4e33cb016560,1,3
...,...,...,...,...,...,...,...,...
440,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,6,188
441,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,0757ae64-497c-40b7-a1db-3d4730e71368,7,188
442,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,1,189
443,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,e7b76451-15e1-4c7e-b84e-a4336fb83654,2,189


In [125]:
owners_matrices = pd.merge(owners_df, testDF, how='left', left_on=['row_index'], right_on=['o_row_index'])
#add a uuid to each distinct group of line with same values in registre, type_folio, folio and groupe_cf
#owners_matrices.to_csv(ROOT + 'owners_matrices.csv',index=False)
owners_matrices

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription,row_index,m_registre,m_type_folio,m_num_folio,m_groupe_cf,m_image,m_uuid,m_row_num_in_cf,o_row_index
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau,0,MAT_1813,Bâti,108,1,FRAD094_3P_000255_01_0586,7ab9a942-8c75-45f2-a04f-7466a762ddbb,1,0
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves,1,MAT_1813,Bâti,114,1,FRAD094_3P_000255_01_0588,afdad904-966f-4ecf-8e09-cdae95dddc28,1,1
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,1,2
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,2,2
4,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...,3,MAT_1813,Bâti,64,1,FRAD094_3P_000255_01_0571,5f26bd83-015b-416c-a62e-4e33cb016560,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,6,188
441,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,0757ae64-497c-40b7-a1db-3d4730e71368,7,188
442,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,1,189
443,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,e7b76451-15e1-4c7e-b84e-a4336fb83654,2,189


In [126]:
#I want to create a dict with o_row_index as key and uuids as values
from datascroller import scroll

cf_uuid_dict = {}
sees = []
for index, row in owners_matrices.iterrows():
    if row['o_row_index'] not in sees:
        cf_uuid_dict[row['o_row_index']] = uuid.uuid4()
        sees.append(cf_uuid_dict[row['o_row_index']])

#pd.set_option("display.max_rows", None)
display(owners_matrices)

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription,row_index,m_registre,m_type_folio,m_num_folio,m_groupe_cf,m_image,m_uuid,m_row_num_in_cf,o_row_index
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau,0,MAT_1813,Bâti,108,1,FRAD094_3P_000255_01_0586,7ab9a942-8c75-45f2-a04f-7466a762ddbb,1,0
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves,1,MAT_1813,Bâti,114,1,FRAD094_3P_000255_01_0588,afdad904-966f-4ecf-8e09-cdae95dddc28,1,1
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,1,2
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1,FRAD094_3P_000255_01_0579,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,2,2
4,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...,3,MAT_1813,Bâti,64,1,FRAD094_3P_000255_01_0571,5f26bd83-015b-416c-a62e-4e33cb016560,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,6,188
441,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1,FRAD094_3P_000264_01_0221,0757ae64-497c-40b7-a1db-3d4730e71368,7,188
442,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,1,189
443,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1,FRAD094_3P_000264_01_0222,e7b76451-15e1-4c7e-b84e-a4336fb83654,2,189


* Créer les comptes fonciers
* Associer à chaque compte foncier ses propriétaires (ordonnés dans le temps)
* Associer à chaque compte foncier le landmark (état) qu'il mentionne

* ```o_row_index``` : id de l'article de mutation (et du compte foncier => 1..1)
donc
    * ArticleDeMutation : uuid_mutation
    * CompteFoncier : uuid
    * Owner : uuid_taxpayer = owner_id

* ```m_uuid``` : id de l'article de classement
donc
    * ArticleDeClassement : uuid

    #Source
    #rowSource = BNode()
    #g.add((subject_uri, cad.sourcedFrom, rowSource))
    #g.add((rowSource, RDF.type, fpo.SourceCitation))
    #g.add((rowSource, fpo.fromSource, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))
    #g.add((rowSource, rico.isComponentOfTransitive, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    #g.add((rowSource, PROV.wasGeneratedBy, URIRef(cad_atype + "0002")))
    #g.add((URIRef(cad_atype + "0002"), PROV.used, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    #g.add((rowSource, rico.isOrWasDigitalInstanciation,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")))
    #g.add((rowSource, cad.lineOrderInArea, Literal(row['Ordre_de_lecture'], datatype=XSD.integer)))

In [127]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('source', srcuri)
g.bind('taxpayer', owneruri)
g.bind('event', eventuri)

g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('cad_etype', cad_etype)
g.bind('ctype', ctype)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)
g.bind('rico', rico)
g.bind('add', add)
g.bind('cad', cad)

created_cf = []

for index, row in owners_matrices.iterrows():
    #Infos de l'image

    json = parse_record_id(row["m_image"])
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

    #UUID du compte foncier
    cfuuid_ = str(cf_uuid_dict[row['o_row_index']])
    subject_uri = URIRef(srcuri + f"{cfuuid_}")
    #UUID de l'instanciation
    Icfuri = URIRef(subject_uri + "instanciation")
    g.add((Icfuri, RDF.type, rico.Instanciation))
    g.add((Icfuri, rico.isOrWasDigitalInstanciation, subject_uri))
    g.add((Icfuri, PROV.wasGeneratedBy, URIRef(activity + "0002")))

    #URI de l'article de classement
    articleclassementuri = URIRef(srcuri + f"{row['m_uuid']}_classement")
    g.add((articleclassementuri, RDF.type, rico.RecordPart))
    g.add((articleclassementuri, cad.isSourceType, URIRef(srctype.ArticleDeClassement)))
    g.add((articleclassementuri, DCTERMS.identifier, Literal(row['m_row_num_in_cf'],datatype=XSD.integer))) #Numéro d'ordre de l'article de classement dans le compte foncier
    #UUID de l'instanciation
    Iarticleclassementuri = URIRef(srcuri + f"{row['m_uuid']}_classement_instanciation")
    g.add((Iarticleclassementuri, RDF.type, rico.Instanciation))
    g.add((Iarticleclassementuri, rico.isOrWasDigitalInstanciation, articleclassementuri))
    g.add((Iarticleclassementuri, PROV.wasGeneratedBy, URIRef(activity + "0002")))

    #CF contient ArticleDeClassement
    g.add((subject_uri, rico.hasOrHadConstituent, articleclassementuri)) 

    if cfuuid_ not in created_cf:
        created_cf.append(cfuuid_)
        #Compte foncier
        g.add((subject_uri, RDF.type, rico.RecordPart))
        g.add((subject_uri, cad.isSourceType, URIRef(srctype.CompteFoncier)))
        g.add((subject_uri, rico.hasOrHadConstituent, URIRef(srcuri + f"{cfuuid_}_mutation"))) #CF contient ArticleDeMutation
        g.add((subject_uri, rico.isOrWasConstituentOf, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['m_num_folio'])}"))) #CF est contenu dans Folio
        g.add((subject_uri, DCTERMS.identifier, Literal(row['groupe_cf'],datatype=XSD.integer))) #Numéro d'ordre du compte foncier dans un folio
        
        #URI de l'article de mutation
        articlemutationuri = URIRef(srcuri + f"{cfuuid_}_mutation")
        g.add((articlemutationuri, RDF.type, rico.RecordPart))
        g.add((articlemutationuri, cad.isSourceType, srctype.ArticleDeMutation))
        #UUID de l'instanciation
        Iarticlemutationuri = URIRef(srcuri + f"{cfuuid_}_mutation_instanciation")
        g.add((Iarticlemutationuri, RDF.type, rico.Instanciation))
        g.add((Iarticlemutationuri, rico.isOrWasDigitalInstanciation, articlemutationuri))
        g.add((Iarticlemutationuri, PROV.wasGeneratedBy, URIRef(activity + "0002")))
        g.add((Iarticlemutationuri, cad.transcription, Literal(row['transcription'],datatype=XSD.string)))

        ## Taxpayers contenu dans ArticleDeMutation
        ownersattribute = BNode()
        g.add((ownersattribute, RDF.type, add.Attribute))
        g.add((articlemutationuri, cad.hasCadastreAttribute, ownersattribute))
        g.add((ownersattribute, add.isAttributeType, cad_atype.PlotTaxpayer))

        owners_and_versions = {}
        for owner in row['owners']:
            ownersattributeversion = BNode()
            g.add((ownersattributeversion,RDF.type,add.AttributeVersion))
            g.add((ownersattribute, add.hasAttributeVersion, ownersattributeversion))
            owneruriinstance = URIRef(owneruri + f"{cfuuid_}_taxpayer_{owner['owner-id']}")
            g.add((owneruriinstance, RDF.type, cad.Taxpayer))
            g.add((ownersattributeversion, cad.hasTaxpayer, owneruriinstance))
            olabel = ''
            #test if json has a certain key
            if 'owner-lastname' in owner:
                g.add((owneruriinstance, cad.taxpayerLabel, Literal(owner['owner-lastname'],datatype=XSD.string)))
                olabel += owner['owner-lastname']
            if 'owner-firstname' in owner:
                g.add((owneruriinstance, cad.taxpayerFirstName, Literal(owner['owner-firstname'],datatype=XSD.string)))
                olabel += ' ' + owner['owner-firstname']
            if 'owner-status' in owner:
                g.add((owneruriinstance, cad.taxpayerStatus, Literal(owner['owner-status'],datatype=XSD.string)))
                olabel += ' (' + owner['owner-status'] + ')'
            if 'owner-activity' in owner:
                g.add((owneruriinstance, cad.taxpayerActivity, Literal(owner['owner-activity'],datatype=XSD.string)))
            if 'owner-address' in owner:
                g.add((owneruriinstance, cad.taxpayerAddress, Literal(owner['owner-address'],datatype=XSD.string)))
            #Create owner label
            g.add((owneruriinstance, RDFS.label, Literal(olabel,datatype=XSD.string)))
            g.add((owneruriinstance,cad.fromSource,articlemutationuri))
            owners_and_versions[owner['owner-id']] = ownersattributeversion

        if len(row) > 0:
            for change in row['changes']:
                changenode = BNode()
                g.add((URIRef(changenode.n3()), RDF.type, add.Change))
                g.add((URIRef(changenode.n3()), add.appliedTo, ownersattribute))
                g.add((URIRef(changenode.n3()), add.outdates, owners_and_versions[change['owner-before']]))
                g.add((URIRef(changenode.n3()), add.makesEffective, owners_and_versions[change['owner-after']]))
                g.add((URIRef(changenode.n3()),add.isChangeType,ctype.AttributeVersionTransition))
                event_uuid = uuid.uuid4()
                event_uri = URIRef(eventuri + f"{event_uuid}")
                g.add((URIRef(changenode.n3()), add.dependsOn, event_uri))
                if 'date' in change:
                    g.add((event_uri, RDF.type, add.Event))
                    time_ = BNode()
                    g.add((event_uri, add.hasTime, time_))
                    g.add((event_uri, cad.isEventType, cad_etype.TaxpayerMutation))
                    g.add((time_, RDF.type, add.TimeInstant))
                    g.add((time_, add.timeCalendar, time.Gregorian))
                    g.add((time_, add.Precision, time.Year))
                    g.add((time_, add.timeStamp, Literal(datetime.strptime(str(int(change['date'])) + '-12-31', '%Y-%m-%d'),datatype=XSD.dateTimeStamp)))
        #Create initial change
        initchangenode = BNode()
        g.add((URIRef(initchangenode.n3()), RDF.type, add.Change))
        g.add((URIRef(initchangenode.n3()), add.appliedTo, ownersattribute))
        g.add((URIRef(initchangenode.n3()), add.makesEffective, owners_and_versions[1]))
        g.add((URIRef(initchangenode.n3()), add.isChangeType, ctype.AttributeVersionAppearance))
        event_uuid = uuid.uuid4()
        event_uri = URIRef(eventuri + f"{event_uuid}")
        g.add((URIRef(initchangenode.n3()), add.dependsOn, event_uri))

        #Create final change
        finalchangenode = BNode()
        g.add((URIRef(finalchangenode.n3()), RDF.type, add.Change))
        g.add((URIRef(finalchangenode.n3()), add.appliedTo, ownersattribute))
        g.add((URIRef(finalchangenode.n3()), add.outdates, owners_and_versions[len(owners_and_versions)]))
        g.add((URIRef(finalchangenode.n3()), add.isChangeType, ctype.AttributeVersionDisappearance))
        event_uuid = uuid.uuid4()
        event_uri = URIRef(eventuri + f"{event_uuid}")
        g.add((URIRef(finalchangenode.n3()), add.dependsOn, event_uri))

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_owners_cf_clas_mut.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/> .
@prefix ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix event: <http://data.ign.fr/id/event/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix taxpayer: <http://data.ign.fr/id/taxpayer/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N0085ae21b1d14248b946d6d521c498ee> a add:Change ;
    add:appliedTo _:Nda82b28629c247cf993e908622a690bb ;
    add:dependsOn ev

<Graph identifier=N3e7695d6e91c4af2a68bb89fe9071031 (<class 'rdflib.graph.Graph'>)>

### 1.X Création des états de parcelles

In [128]:
matrices

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe_CF,Ordre_de_lecture,Voie,Num_Voie,Image,...,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean
0,1,7ab9a942-8c75-45f2-a04f-7466a762ddbb,Bâti,108,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0586,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,108,EMPTY,EMPTY
1,2,afdad904-966f-4ecf-8e09-cdae95dddc28,Bâti,114,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0588,...,82,Oui,Oui,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,114,EMPTY,82
2,3,9db66a5a-b0f4-4156-9cd1-9fe730175f0b,Bâti,82,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,114,EMPTY
3,4,24d96d36-a7c1-4deb-a0a2-130f8a1649ba,Bâti,82,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,EMPTY,EMPTY
4,5,5f26bd83-015b-416c-a62e-4e33cb016560,Bâti,64,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0571,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,64,EMPTY,EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,219,4a1990b8-03a7-4cd8-bf8a-4eb8c6441b4d,Non Bâti,1221,NaN,1,6,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
438,220,0757ae64-497c-40b7-a1db-3d4730e71368,Non Bâti,1221,NaN,1,7,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
439,221,d6e8d97b-264f-4aba-b26e-d60d48d7a12f,Non Bâti,1222,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY
440,222,e7b76451-15e1-4c7e-b84e-a4336fb83654,Non Bâti,1222,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY


In [129]:
matrices.columns

Index(['ID', 'UUID', 'Type_CF', 'Num_Folio', 'Alt_Num_CF', 'Groupe_CF',
       'Ordre_de_lecture', 'Voie', 'Num_Voie', 'Image', 'Section_clean',
       'Parcelle_clean', 'Parcelle_treated', 'Lieu-dit_transcript',
       'Lieu-dit_clean', 'Lieu-dit_treated', 'Lieu-dit_type',
       'Propriétaires_transcript', 'Nature_transcript', 'Nature_clean',
       'Nature_treated', 'Date entrée', 'Date entrée_treated', 'Date sortie',
       'Date sortie_treated', 'Tiré de', 'Tiré de_treated', 'Porté à',
       'Porté à_treated', 'Ligne barrée ?', 'CF rayé ?', 'Spécification',
       'Commentaire', 'Cote liée', 'registre', 'Num_Folio_clean',
       'Tire_de_clean', 'Porte_a_clean'],
      dtype='object')

In [130]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('cad_etype', cad_etype)
g.bind('cad_spval', cad_spval)
g.bind('ctype', ctype)
g.bind('lrtype', lrtype)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)
g.bind('pnature', pnature)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in matrices.iterrows():
    #Infos de l'image
    json = parse_record_id(row["Image"])
    #Infos de la matrice et du plan
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
    MATRICE_START = matrices_metada[row['registre']]["MATRICE_START"]
    MATRICE_END = matrices_metada[row['registre']]["MATRICE_END"]
    PLAN = matrices_metada[row['registre']]["PLAN"]
    #Article de classement correspondant à la ligne
    articleclassementuri = URIRef(srcuri + f"{row['UUID']}_classement")

    #Détail de l'extraction (transcriptions brutes)
    Iarticleclassementuri = URIRef(srcuri + f"{row['UUID']}_classement_instanciation")
    g.add((Iarticleclassementuri, cad.lineOrderInArea, Literal(row['Ordre_de_lecture'], datatype=XSD.integer))) #Ordre dans le compte foncier (ce sera plutôt dans la page avec DAN)
    if not pd.isnull((row["Section_clean"])):
        sectionT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, sectionT))
        g.add((sectionT, RDF.type, cad.NamedEntity))
        g.add((sectionT, cad.isNamedEntityType, mlclasse.SectionIdentifier))
        g.add((sectionT, cad.transcription, Literal(row["Section_clean"],datatype=XSD.string)))
    if not pd.isnull((row["Parcelle_clean"])):
        plotnumberT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, plotnumberT))
        g.add((plotnumberT, RDF.type, cad.NamedEntity))
        g.add((plotnumberT, cad.isNamedEntityType, mlclasse.PlotNumber))
        g.add((plotnumberT, cad.transcription, Literal(row["Parcelle_clean"],datatype=XSD.string)))
    if not pd.isnull((row["Lieu-dit_transcript"])):
        districtT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, districtT))
        g.add((districtT, RDF.type, cad.NamedEntity))
        g.add((districtT, cad.isNamedEntityType, mlclasse.NamedPlace))
        g.add((districtT, cad.transcription, Literal(row["Lieu-dit_transcript"],datatype=XSD.string)))
    if not pd.isnull((row["Nature_transcript"])):
        natureT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, natureT))
        g.add((natureT, RDF.type, cad.NamedEntity))
        g.add((natureT, cad.isNamedEntityType, mlclasse.Nature))
        g.add((natureT, cad.transcription, Literal(row["Nature_transcript"],datatype=XSD.string)))
    if not pd.isnull((row["Date entrée"])):
        dateT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, dateT))
        g.add((dateT, RDF.type, cad.NamedEntity))
        g.add((dateT, cad.isNamedEntityType, mlclasse.StartTime))
        g.add((dateT, cad.transcription, Literal(row["Date entrée"],datatype=XSD.string)))
    if not pd.isnull((row["Date sortie"])):
        dateTS = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, dateTS))
        g.add((dateTS, RDF.type, cad.NamedEntity))
        g.add((dateTS, cad.isNamedEntityType, mlclasse.EndTime))
        g.add((dateTS, cad.transcription, Literal(row["Date sortie"],datatype=XSD.string)))
    if not pd.isnull((row["Tiré de"])):
        takenfromT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, takenfromT))
        g.add((takenfromT, RDF.type, cad.NamedEntity))
        g.add((takenfromT, cad.isNamedEntityType, mlclasse.TakenFrom))
        g.add((takenfromT, cad.transcription, Literal(row["Tiré de"],datatype=XSD.string)))
    if not pd.isnull((row["Porté à"])):
        passedtoT = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasNamedEntity, passedtoT))
        g.add((passedtoT, RDF.type, cad.NamedEntity))
        g.add((passedtoT, cad.isNamedEntityType, mlclasse.PassedTo))
        g.add((passedtoT, cad.transcription, Literal(row["Porté à"],datatype=XSD.string)))
    if not pd.isnull((row["Ligne barrée ?"])) and row["Ligne barrée ?"] == 'Oui':
        styleclasse = URIRef(BNode().n3())
        g.add((Iarticleclassementuri, cad.hasClasse, styleclasse))
        g.add((styleclasse, cad.hasClasseValue, mlclasse.CrossedOut))

    #Création de l'état des landmarks de type Plot
    subject_uri = URIRef(landmarkuri + f"{row['UUID']}")
    g.add((subject_uri, RDF.type, add.Landmark))
    g.add((subject_uri, add.isLandmarkType, cad_ltype.Plot))
    if ';' in row['Parcelle_treated']:
        nums = row['Parcelle_treated'].split(';')
        label = f" {row['Section_clean']}-".join(nums)
        for n in nums:
            g.add((subject_uri, DCTERMS.identifier, Literal(row['Section_clean'] + '-' + str(n), datatype=XSD.string)))
    else:
        label = row['Parcelle_treated']
        g.add((subject_uri, DCTERMS.identifier, Literal(row['Section_clean'] + '-' + row['Parcelle_treated'], datatype=XSD.string)))
    g.add((subject_uri, SKOS.prefLabel, Literal(row['Section_clean'] + '-' + str(label) + ' ('+ COMMUNE +')', datatype=XSD.string)))
    g.add((subject_uri, RDFS.label, Literal(row['Section_clean'] + '-' + str(label) + ' ('+ COMMUNE +')', datatype=XSD.string)))

    #LandmarkRelation certaine Plot <=> Section
    relationode = URIRef(BNode().n3())
    g.add((relationode, RDF.type, add.LandmarkRelation))
    g.add((relationode, add.isLandmarkRelationType, lrtype.Within))
    g.add((relationode, add.locatum, subject_uri))
    if row['registre'] == 'MAT_1848':
        g.add((relationode, add.relatum, URIRef(landmarkuri + 'da6a5c2c-e86d-43bb-8950-7169bd0df60a')))
    else:
        g.add((relationode, add.relatum, URIRef(landmarkuri + '87d7c2f6-306b-45a1-a833-5e17821c3102')))

    #Article de classement et folios précédents/suivants
    plotMentionAttr = URIRef(BNode().n3())
    g.add((subject_uri, add.hasAttribute,plotMentionAttr))
    g.add((plotMentionAttr, RDF.type, add.Attribute))
    g.add((plotMentionAttr, add.isAttributeType, cad_atype.PlotMention))
    plotMentionAttrVersion = URIRef(BNode().n3())
    g.add((plotMentionAttr, add.hasAttributeVersion, plotMentionAttrVersion))
    g.add((plotMentionAttrVersion, RDF.type, add.AttributeVersion))
    
    ## Article de classement
    g.add((plotMentionAttrVersion, cad.isMentionnedIn, articleclassementuri))

    ## Folio.s ou valeur.s spéciale.s précédent.s (Tiré de)
    if not pd.isnull(row['Tiré de_treated']):
        tire_de = str(row['Tiré de_treated']).split(';')
        for f in tire_de:
            f = str(f)
            if any(char.isdigit() for char in f) and 'mission' not in f:
                g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{f}")))
            elif 'mission' in f:
                g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(cad_spval + f"OmissionSV")))
            elif f != '' or f != 'nan':
                g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(cad_spval + f)))
    else:
        g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(cad_spval + "CelluleVideSV")))
    ## Folio.s ou valeur.s spéciale.s  suivant.s (Porté à)
    if not pd.isnull(row['Porté à_treated']):
        porte_a = str(row['Porté à_treated']).split(';')
        for f in porte_a:
            if any(char.isdigit() for char in str(f)) and 'mission' not in f:
                g.add((plotMentionAttrVersion, cad.passedTo, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{f}")))
            elif f != '' or f != 'nan':
                g.add((plotMentionAttrVersion, cad.passedTo, URIRef(cad_spval + f)))
    else:
        g.add((plotMentionAttrVersion, cad.passedTo, URIRef(cad_spval + "CelluleVideSV")))

    ## Time
    hastime = URIRef(BNode().n3())
    if not pd.isnull(row['Date entrée_treated']) or not pd.isnull(['Date sortie_treated']):
        g.add((plotMentionAttrVersion, add.hasTime, hastime))
        g.add((hastime, RDF.type, add.TimeInterval))

        hasbeginning = URIRef(BNode().n3())
        g.add((hasbeginning, RDF.type, add.TimeInstant))
        g.add((hasbeginning, add.timeCalendar, time.Gregorian))
        g.add((hasbeginning, add.timePrecision, time.Year))
        if not pd.isna(row['Date entrée_treated']) and not pd.isnull(row['Date entrée_treated']):
            g.add((hastime, add.hasBeginning, hasbeginning))
            g.add((hasbeginning, add.timeStamp, Literal(datetime.strptime(str(int(row['Date entrée_treated'])) + '-01-01', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))
        else:
            g.add((plotMentionAttrVersion, add.hasEarliestTimeInstant, hasbeginning))
            g.add((hasbeginning, add.timeStamp, Literal(datetime.strptime(MATRICE_START + '-01-01', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))#date d'ouverture de la matrice
        
        hasend = URIRef(BNode().n3())
        g.add((hasend,RDF.type, add.TimeInstant))
        g.add((hasend, add.timeCalendar, time.Gregorian))
        g.add((hasend, add.timePrecision, time.Year))

        if not pd.isnull(row['Date sortie_treated']) and row['Date sortie_treated'] != 'nan':
            g.add((hastime, add.hasEnd, hasend))
            g.add((hasend, add.timeStamp, Literal(datetime.strptime(str(int(row['Date sortie_treated'])) + '-12-31', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))
        else:
            g.add((plotMentionAttrVersion, add.hasLatestTimeInstant, hasend))
            g.add((hasend, add.timeStamp, Literal(datetime.strptime(MATRICE_END + '-12-31', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))
    
    #Address
    if not pd.isnull(row['Lieu-dit_treated']):
        plotaddress = URIRef(BNode().n3())
        g.add((subject_uri, add.hasAttribute, plotaddress))
        g.add((plotaddress, RDF.type, add.Attribute))
        g.add((plotaddress, add.isAttributeType, cad_atype.PlotAddress))
        plotaddressversion = URIRef(BNode().n3())
        g.add((plotaddress, add.hasAttributeVersion, plotaddressversion))
        g.add((plotaddressversion, RDF.type, add.AttributeVersion))
        relationode = URIRef(BNode().n3())
        g.add((plotaddressversion, cad.hasPlotAddress, relationode))
        g.add((relationode, RDF.type, add.LandmarkRelation))
        g.add((relationode, add.isLandmarkRelationType, lrtype.Undefined))
        g.add((relationode, add.locatum, subject_uri))
        for indexadd, rowadd in addresses.iterrows():
            if row['Lieu-dit_treated'] == rowadd['Lieu-dit_treated'] and row['registre'] == rowadd['registre']:
                g.add((relationode, add.relatum, URIRef(landmarkuri + str(rowadd['address_uuid']))))

    #Nature
    if not pd.isnull(row['Nature_treated']):
        nature = URIRef(BNode().n3())
        g.add((subject_uri, add.hasAttribute, nature))
        g.add((nature, RDF.type, add.Attribute))
        g.add((nature, add.isAttributeType, cad_atype.PlotNature))
        natures = row['Nature_treated'].split(',')
        natureversion = URIRef(BNode().n3())
        g.add((natureversion, RDF.type, add.AttributeVersion))
        g.add((nature, add.hasAttributeVersion, natureversion))
        for n in natures:
            g.add((natureversion, add.hasNature, URIRef(pnature + n)))

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_landmarks_plot_mentions.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/> .
@prefix cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix pnature: <http://data.ign.fr/id/codes/cadastre/plotNature/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N013af40c83c246f2b2ff699ef22a9164> a add:LandmarkRelation ;
  

<Graph identifier=Nae0ea8512d094fc3ba23b84bceceb300 (<class 'rdflib.graph.Graph'>)>